In [226]:
library(ggplot2)
library(ggrastr)
library(ggrepel)
library(tidyverse)
library(scales)

data <- read.csv("../data/DMS/antibody/_clustering.csv")

cb_palette <- c(
    "A1"="#004400","A2"="#237732","B"="#CC3119",
    #"C/D1"="#ffd700",
    "D2"="#33aa11", "D3"="#99AA11","D4"="#80A07A",
    "E1/E2.1"="#664088", "E2.2"='#aa1937', 
    # "E1/E2"='#aa1937', 
    "E3"="#5c0a02", 
    "F1.1"="#c58233", "F1.2"="#253257",
    "F3"="#7093c8"
)

label_mAbs <- c(
    # "GC2050" = "BD57-2050", 
    "GC2225" = "BD57-2225", 
    # "GC1520" = "BD57-1520", 
    # "GC5165" = "GC5165", 
    "BD45-1" = "S309", 
    "BD45-14" = "S304", 
    "BD45-105" = "S2H97", 
    "BD45-117" = "FC08", 
    "BD55-1205" = "BD55-1205",
    "BD55-5514" = "SA55",
    "VYD222" = "VYD222"
)

sources <-  c("XBB infection", "XBB BTI", "BA.5 + XBB infection", "BA.5 BTI + XBB infection", "BA.5 BTI + HK.3 infection", "BA.5 + JN.1 infection", "BA.5 BTI + JN.1 infection")

data$show_name <- label_mAbs[data$id]
data$assayed <- c('0'='JN.1 only', '1'='both', '2'='XBB.1.5 only')[as.character((data$XBB15_dms_assayed == 'True') + (data$JN1_dms_assayed == 'False'))]
data$plot_src <- ifelse(data$source %in% sources, data$source, "others")

dev.off()
pdf("../plots/Fig2/DMS-UMAP.pdf", height=3, width=4)
ggplot(data, aes(UMAP1, UMAP2))+geom_point_rast(size=.5, alpha=0.7, aes(color=new_group))+
    theme_classic()+
    geom_point(data=data %>% filter(id %in% names(label_mAbs)), shape=21, size=2)+
    geom_text_repel(data=data %>% filter(id %in% names(label_mAbs)), aes(label=show_name), min.segment.length = Inf)+
    theme(aspect.ratio = 1.0,
          panel.border = element_rect(colour = "black", fill=NA, size=1),
          axis.line = element_blank(),
          axis.ticks = element_blank(),
          axis.text = element_blank(),
         ) + 
    scale_color_manual(values=cb_palette) + 
    xlab("UMAP1") + ylab("UMAP2")+ guides(color=guide_legend(title="Epitope\nGroup", override.aes = list(size=2,alpha=1)))

cl_label <- data %>% group_by(cluster) %>% summarise(x=mean(UMAP1), y=mean(UMAP2))

dev.off()

pdf("../plots/Extended/DMS/DMS-UMAP-ext.pdf", height=3, width=4)
ggplot(data, aes(UMAP1, UMAP2))+geom_point_rast(size=.5, alpha=0.7, aes(color=cluster), show.legend=F)+
    geom_text(data=cl_label, aes(x,y, label=cluster))+
    theme_classic()+
    theme(aspect.ratio = 1.0,
          panel.border = element_rect(colour = "black", fill=NA, size=1),
          axis.line = element_blank(),
          axis.ticks = element_blank(),
          axis.text = element_blank(),
         ) + 
    xlab("UMAP1") + ylab("UMAP2")#+ guides(color=guide_legend(title="Cluster", override.aes = list(size=2,alpha=1)))

ggplot(data, aes(UMAP1, UMAP2))+geom_point_rast(size=.5, alpha=0.5, aes(color=assayed))+
    theme_classic()+
    theme(aspect.ratio = 1.0,
          panel.border = element_rect(colour = "black", fill=NA, size=1),
          axis.line = element_blank(),
          axis.ticks = element_blank(),
          axis.text = element_blank(),
         ) + 
    xlab("UMAP1") + ylab("UMAP2")+ guides(color=guide_legend(title="Assayed", override.aes = list(size=2,alpha=1)))

for (src in unique(data$plot_src)) {
    print(ggplot(data%>% filter(plot_src != src), aes(UMAP1, UMAP2))+
          geom_point_rast(size=.3, alpha=0.7, color="#DDDDDD")+
          geom_point_rast(data=data%>%filter(plot_src == src), size=.6, alpha=0.7, color="blue")+
        theme_classic()+
        theme(aspect.ratio = 1.0,
              panel.border = element_rect(colour = "black", fill=NA, size=1),
              axis.line = element_blank(),
              axis.ticks = element_blank(),
              axis.text = element_blank(),
             ) + 
        xlab("UMAP1") + ylab("UMAP2")+ggtitle(src)
    )
}


ggplot(data[!is.na(data$ACE2_competition),], aes(UMAP1,UMAP2))+ggtitle("ACE2 Competition Level")+
    geom_point_rast(size=1, alpha=0.5, aes(color=ACE2_competition))+theme_classic()+
    theme(aspect.ratio = 1.0,panel.border = element_rect(colour = "black", fill=NA, size=1),
          axis.line = element_blank(),
          plot.title = element_text(hjust = 0.5,size=15,face="bold"),
          axis.ticks = element_blank(),
          axis.text = element_blank()) + 
    coord_fixed()+xlab("")+ylab("")+
    scale_color_gradient(low="#dddddd",high="#792060", limits=c(0,1), name="", oob=squish)

dev.off()


plot_ic50_strains <- c("D614G_IC50", "BA1_IC50", "BA2_IC50", "BA5_IC50", "XBB1_5_IC50","JN1_IC50","JN1_R346T_F456L_IC50","KP3_IC50"
                      )
pdf("../plots/Extended/DMS/antibody/Abs_info_maps.pdf", width=4, height=3)

for (strain in plot_ic50_strains) {
p <- ggplot(data[!is.na(data[,strain]),], aes(UMAP1,UMAP2))+geom_point_rast(size=.5, alpha=0.2, aes(color=get(strain)), show.legend=F)+
    theme_classic()+coord_fixed()+xlab("")+ylab("")+ggtitle(strain)+
    theme(aspect.ratio = 1.0,panel.border = element_rect(colour = "black", fill=NA, size=1),
          axis.line = element_blank(),
          plot.title = element_text(hjust = 0.5,size=15,face="bold"),
          axis.ticks = element_blank(),
          axis.text = element_blank()) + scale_color_gradient2(low="yellow",mid="red",high="#888888", midpoint=-2, 
                                                      limits=c(0.001,10), oob=squish, trans="log", breaks=c(0.001, 0.1, 10),
                                                      labels = c('0.001', '0.1', '10'), name="")
print(p)
}

dev.off()

pdf 
  3

pdf 
  3

pdf 
  3

pdf 
  3

In [227]:
data[,c("id", "UMAP1","UMAP2",'ACE2_competition', plot_ic50_strains)] %>% pivot_longer(!id) %>% na.omit() %>% group_by(name) %>% summarise(count=n())

name,count
<chr>,<int>
ACE2_competition,1565
BA1_IC50,2684
BA2_IC50,2682
BA5_IC50,2686
D614G_IC50,2686
JN1_IC50,2481
JN1_R346T_F456L_IC50,2069
KP3_IC50,2469
UMAP1,2688


In [228]:

group_levels <- c("F3", "A1", "A2", "B", "D2", "D3", "D4", "E1/E2.1", "E2.2", "E3", "F1.1", "F1.2")
src_levels = c(
    'XBB infection',
    'XBB BTI',
    'BA.5 + XBB infection',
    'BA.5 + JN.1 infection',
    'BA.5 BTI + XBB infection',
    'BA.5 BTI + HK.3 infection', 
    'BA.5 BTI + JN.1 infection'
)
bar_df <- read.csv("../data/DMS/antibody/_group_stat.csv") %>% filter(source %in% src_levels)

bar_df$group <- factor(bar_df$new_group, levels=group_levels)
bar_df$source <- factor(bar_df$source, levels=src_levels)

dev.off()
pdf('../plots/Figure3_related/groups_count_bar.pdf',width=4,height=4)

ggplot(bar_df, aes(source, sum*100)) + geom_bar(aes(fill=group), stat='identity', width=0.75, color='black') + 
    scale_fill_manual(values=cb_palette) + 
    theme_classic() + 
    scale_y_continuous(breaks=c(0:10)*10, expand = c(0,0))+
    geom_text(data=bar_df %>% group_by(source) %>% summarise(n=sum(count)), aes(x=source, y=105, label=n), size=3)+
    theme(axis.title = element_blank(), 
          axis.text.x=element_text(angle=90,hjust=1,vjust=0.5), axis.line.x = element_blank(), 
          axis.ticks.x = element_blank())

dev.off()

pdf 
  3

pdf 
  3

In [229]:
dev.off()
src_levels = c(
    'XBB infection',
    'XBB BTI',
    'BA.5 + XBB infection',
    'BA.5 + JN.1 infection',
    'BA.5 BTI + XBB infection',
    'BA.5 BTI + HK.3 infection', 
    'BA.5 BTI + JN.1 infection'
)
dir = c(
    'all' = 'Extended/DMS/antibody/',
    'cross' = 'Figure4_related/',
    'specific' = 'Figure5_related/'
)

bar_df_all <- read.csv("../data/DMS/antibody/_neut_weight_group_stat.csv") %>% filter(source %in% src_levels)
for (prefix in c('all', 'cross', 'specific')) {
    pdf(paste('../plots/',dir[prefix],prefix, '_groups_bar_neut_weighting_all.pdf', sep=''),width=2,height=3)
    M <- max((bar_df_all %>% filter(use_mAbs == prefix) %>% group_by(source, weight) %>% summarise(e=sum(effectiveness)))$e)
    for (use_neut in c("BA5_IC50", "XBB1_5_IC50","HK3_1_IC50",  "JN1_IC50", "JN1_R346T_F456L_IC50", "KP3_IC50")) {
        
        bar_df <- bar_df_all %>% filter(weight == use_neut & use_mAbs == prefix)
        bar_df$source <- factor(bar_df$source, levels=src_levels)
        bar_df$group <- factor(bar_df$new_group, levels=group_levels)

        p <- ggplot(bar_df, aes(source, effectiveness)) + geom_bar(aes(fill=group), stat='identity', width=0.8, color='black', show.legend=F) + 
            scale_fill_manual(values=cb_palette) + 
            theme_classic() + 
            scale_y_continuous(expand=c(0,0), limits=c(0, M))+ylab('Effectiveness')+
            theme(axis.title = element_blank(),
                  axis.text.x=element_text(angle=45,hjust=1),  
                  axis.ticks.x = element_blank(), legend.position='top')+
            ggtitle(use_neut)
        print(p)
    }
    dev.off()
}

src_levels = c(
    'BA.5 BTI + XBB infection',
    'BA.5 BTI + HK.3 infection', 
    'BA.5 BTI + JN.1 infection'
)
bar_df_all <- read.csv("../data/DMS/antibody/_neut_weight_group_stat.csv") %>% filter(source %in% src_levels)
for (prefix in c('all', 'cross', 'specific')) {
    pdf(paste('../plots/',dir[prefix],prefix, '_groups_bar_neut_weighting.pdf', sep=''),width=1.5,height=3)
    M <- max((bar_df_all %>% filter(use_mAbs == prefix) %>% group_by(source, weight) %>% summarise(e=sum(effectiveness)))$e)
    for (use_neut in c("XBB1_5_IC50","HK3_1_IC50",  "JN1_IC50", "JN1_R346T_F456L_IC50", "KP3_IC50")) {
        
        bar_df <- bar_df_all %>% filter(weight == use_neut & use_mAbs == prefix)
        bar_df$source <- factor(bar_df$source, levels=src_levels)
        bar_df$group <- factor(bar_df$new_group, levels=group_levels)

        p <- ggplot(bar_df, aes(source, effectiveness)) + geom_bar(aes(fill=group), stat='identity', width=0.8, color='black', show.legend=F) + 
            scale_fill_manual(values=cb_palette) + 
            theme_classic() + 
            scale_y_continuous(expand=c(0,0), limits=c(0, M))+ylab('Effectiveness')+
            theme(axis.title = element_blank(),
                  axis.text.x=element_text(angle=45,hjust=1),  
                  axis.ticks.x = element_blank(), legend.position='top')+
            ggtitle(use_neut)
        print(p)
    }
    dev.off()
}

pdf 
  3

`summarise()` has grouped output by 'source'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'source'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'source'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'source'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'source'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'source'. You can override using the `.groups` argument.


In [239]:
library(ggrastr)
library(ggrepel)
library(tidyverse)
dev.off()
pdf("../plots/Extended/A1_broad_dms_compare.pdf", width=3.5, height=3)

df <- read.csv("../data/DMS/antibody/_A1_breadth_DMS_scores_compare_avg_XBB15.csv", check.names = F)
print(colnames(df))

colnames(df) <- c('site', 'mutation', 'broad','escaped','site_mut')

ggplot(df, aes(broad, escaped))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(455, 456, 475, 493)), aes(color=factor(site)), size=1.5, shape=21)+
    scale_color_manual(values=c("#EE6439", "#60AB59", "#FFCF56", '#AA2055',"#7193E2"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlim(0,1)+ylim(0,1)+xlab('broad A1')+ylab('escaped A1')+
    geom_text_repel(data=df%>%filter(site_mut %in% c('455S', '456L', '475V', '493E')), aes(label=site_mut), min.segment.length = 0)+
    theme_classic()+theme(aspect.ratio = 1)+ggtitle("XBB.1.5 library DMS")


df <- read.csv("../data/DMS/antibody/_A1_breadth_DMS_scores_compare_avg_JN1.csv", check.names = F)
print(colnames(df))

colnames(df) <- c('site', 'mutation', 'broad','escaped','site_mut')

ggplot(df, aes(broad, escaped))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(455, 456, 475, 493)), aes(color=factor(site)), size=1.5, shape=21)+
    scale_color_manual(values=c("#EE6439", "#60AB59", "#FFCF56", '#AA2055',"#7193E2"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlim(0,1)+ylim(0,1)+xlab('broad A1')+ylab('escaped A1')+
    geom_text_repel(data=df%>%filter(site_mut %in% c('455S', '456L', '475V', '493E')), aes(label=site_mut), min.segment.length = 0)+
    theme_classic()+theme(aspect.ratio = 1)+ggtitle("JN.1 library DMS")


dev.off()


pdf 
  3

[1] "site"         "mutation"     "broad (22)"   "escaped (80)" "site_mut"    
[1] "site"         "mutation"     "broad (29)"   "escaped (54)" "site_mut"    


Warning message:
“Removed 5 rows containing missing values or values outside the scale range (`geom_point()`).”


pdf 
  3

In [236]:
pdf("../plots/Extended/A1_library_dms_compare.pdf", width=3, height=2.5)
df <- read.csv("../data/DMS/antibody/both_mAbs_grouped/A1_antigen.csv") %>% pivot_wider(id_cols=c(site, mutation), names_from=antigen, values_from = mut_escape)
colnames(df) <- c('site', 'mutation', 'JN.1 library DMS', 'XBB.1.5 library DMS')

ggplot(df, aes(`XBB.1.5 library DMS`, `JN.1 library DMS`))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(421, 455, 480, 485, 487, 488)), aes(color=factor(site)), size=1.5, shape=21)+
    scale_color_manual(values=c("#EE6439", "#60AB59", "#FFCF56", '#AA2055',"#7193E2","#33A2B0"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlim(0,1)+ylim(0,1)+
    theme_classic()+theme(aspect.ratio = 1)

df <- read.csv("../data/DMS/antibody/both_mAbs_grouped/A1_antigen.csv") %>% 
    group_by(site, antigen) %>% summarise(mut_escape=mean(mut_escape)) %>% pivot_wider(id_cols=site, names_from=antigen, values_from = mut_escape)
colnames(df) <- c('site', 'JN.1 library DMS', 'XBB.1.5 library DMS')

ggplot(df, aes(`XBB.1.5 library DMS`, `JN.1 library DMS`))+geom_point_rast(size=0.5, alpha=0.8, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(421, 455, 480, 485, 487, 488, 506)), color='black', size=1.5, shape=21)+
    geom_text_repel(data=df%>%filter(site %in% c(421, 455, 480, 485, 487, 488, 506)), color='black', min.segment.length = Inf, aes(label=site), shape=21)+

    # scale_color_manual(values=c("#EE6439", "#60AB59", "#FFCF56", '#AA2055',"#7193E2","#33A2B0", "#66ffcc"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlim(0,0.6)+ylim(0,0.6)+
    theme_classic()+theme(aspect.ratio = 1)


dev.off()

Warning message:
“Removed 441 rows containing missing values or values outside the scale range (`geom_point()`).”
Warning message:
“Removed 7 rows containing missing values or values outside the scale range (`geom_point()`).”
`summarise()` has grouped output by 'site'. You can override using the `.groups` argument.
Warning message in geom_text_repel(data = df %>% filter(site %in% c(421, 455, 480, :
“Ignoring unknown parameters: `shape`”
Warning message:
“Removed 8 rows containing missing values or values outside the scale range (`geom_point()`).”


png 
  2

In [232]:
library(ggrastr)
library(ggrepel)
library(tidyverse)
dev.off()
pdf("../plots/Extended/B_reactivity_dms_compare.pdf", width=3.5, height=3)
df <- read.csv("../data/DMS/antibody/_B_reactivity_DMS_scores_compare.csv")
print(colnames(df))
ggplot(df, aes(cross...JN.1_RBD..9., specific...JN.1_RBD..37.))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(478,486)), aes(color=factor(site)), size=1.5, shape=21)+
    scale_color_manual(values=c("#7193E2", "#B57EDC"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlab('B (cross)')+ylab('B (specific)')+xlim(0,1)+ylim(0,1)+
    # geom_text_repel(data=df%>%filter(site %in% c(455, 456, 475, 493)), aes(label=site_mut))+
    theme_classic()+theme(aspect.ratio = 1)+ggtitle('JN.1 library')
ggplot(df, aes(cross...XBB.1.5_RBD..8., specific...XBB.1.5_RBD..24.))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(478,486)), aes(color=factor(site)), size=1.5, shape=21)+
    scale_color_manual(values=c("#7193E2", "#B57EDC"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlab('B (cross)')+ylab('B (specific)')+xlim(0,1)+ylim(0,1)+
    # geom_text_repel(data=df%>%filter(site %in% c(455, 456, 475, 493)), aes(label=site_mut))+
    theme_classic()+theme(aspect.ratio = 1)+ggtitle('XBB.1.5 library')
dev.off()
pdf("../plots/Extended/D3_reactivity_dms_compare.pdf", width=3.5, height=3)
df <- read.csv("../data/DMS/antibody/_D3_reactivity_DMS_scores_compare.csv")
print(colnames(df))
ggplot(df, aes(cross...JN.1_RBD..13., specific...JN.1_RBD..46.))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(440,444)), aes(color=factor(site)), size=1.5, shape=21)+
    scale_color_manual(values=c("#7193E2", "#B57EDC"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlab('D3 (cross)')+ylab('D3 (specific)')+xlim(0,1)+ylim(0,1)+
    # geom_text_repel(data=df%>%filter(site %in% c(455, 456, 475, 493)), aes(label=site_mut))+
    theme_classic()+theme(aspect.ratio = 1)+ggtitle('JN.1 library')
ggplot(df, aes(cross...XBB.1.5_RBD..11., specific...XBB.1.5_RBD..32.))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(440,444)), aes(color=factor(site)), size=1.5, shape=21)+
    scale_color_manual(values=c("#7193E2", "#B57EDC"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlab('D3 (cross)')+ylab('D3 (specific)')+xlim(0,1)+ylim(0,1)+
    # geom_text_repel(data=df%>%filter(site %in% c(455, 456, 475, 493)), aes(label=site_mut))+
    theme_classic()+theme(aspect.ratio = 1)+ggtitle('XBB.1.5 library')
dev.off()


pdf 
  3

[1] "site"                        "mutation"                   
[3] "cross...JN.1_RBD..9."        "cross...XBB.1.5_RBD..8."    
[5] "specific...JN.1_RBD..37."    "specific...XBB.1.5_RBD..24."
[7] "site_mut"                   


Warning message:
“Removed 342 rows containing missing values or values outside the scale range (`geom_point()`).”
Warning message:
“Removed 106 rows containing missing values or values outside the scale range (`geom_point()`).”


pdf 
  3

[1] "site"                        "mutation"                   
[3] "cross...JN.1_RBD..13."       "cross...XBB.1.5_RBD..11."   
[5] "specific...JN.1_RBD..46."    "specific...XBB.1.5_RBD..32."
[7] "site_mut"                   


Warning message:
“Removed 336 rows containing missing values or values outside the scale range (`geom_point()`).”
Warning message:
“Removed 105 rows containing missing values or values outside the scale range (`geom_point()`).”
Warning message:
“Removed 8 rows containing missing values or values outside the scale range (`geom_point()`).”


pdf 
  3

In [233]:
library(ggrastr)
library(ggrepel)
library(tidyverse)
dev.off()
pdf("../plots/Figure5_related/F3_VH_dms_compare.pdf", width=3.5, height=3)
df <- read.csv("../data/DMS/antibody/_F3_VH_avg_dms.csv") %>% pivot_wider(id_cols=c(site,mutation), names_from=prop, values_from = mut_escape)
colnames(df) <- c('site', 'mutation', 'VH2_5_JN1', 'VH2_5_XBB15', 'VH5_51_JN1', 'VH5_51_XBB15')

ggplot(df, aes(VH2_5_JN1, VH5_51_JN1))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(403,404,405,498,502,504,505)), aes(color=factor(site)), size=1.5, shape=21)+
    scale_color_manual(values=c("#EE6439", "#60AB59", "#FFCF56", '#AA2055',"#7193E2","#33A2B0", "#66ffcc"))+
    geom_abline(linetype='dashed', color='#CC1210')+xlim(0,1)+ylim(0,1)+xlab('IGHV2-5 F3')+ylab('IGHV5-51 F3')+
    theme_classic()+theme(aspect.ratio = 1)

df <- read.csv("../data/DMS/antibody/_F3_VH_avg_dms.csv")  %>% 
    group_by(site, prop) %>% summarise(mut_escape=sum(mut_escape)) %>% pivot_wider(id_cols=site, names_from=prop, values_from = mut_escape)
colnames(df) <- c('site', 'VH2_5_JN1', 'VH2_5_XBB15', 'VH5_51_JN1', 'VH5_51_XBB15')

ggplot(df, aes(VH2_5_JN1, VH5_51_JN1))+geom_point_rast(size=0.5, alpha=0.5, color='#dddddd')+
    geom_point(data=df%>%filter(site %in% c(404,405,498,502,504,505)), color='black', size=1.5, shape=21)+
    geom_text_repel(data=df%>%filter(site %in% c(404,405,498,502,504,505)), color='black', min.segment.length = Inf, aes(label=site), shape=21)+
    geom_abline(linetype='dashed', color='#CC1210')+xlab('IGHV2-5 F3')+ylab('IGHV5-51 F3')+
    theme_classic()+theme(aspect.ratio = 1)
dev.off()


pdf 
  3

Warning message:
“Removed 342 rows containing missing values or values outside the scale range (`geom_point()`).”
Warning message:
“Removed 20 rows containing missing values or values outside the scale range (`geom_point()`).”
`summarise()` has grouped output by 'site'. You can override using the `.groups` argument.
Warning message in geom_text_repel(data = df %>% filter(site %in% c(404, 405, 498, :
“Ignoring unknown parameters: `shape`”
Warning message:
“Removed 7 rows containing missing values or values outside the scale range (`geom_point()`).”


pdf 
  3